In [1]:
from __future__ import print_function, division

from keras.datasets import mnist
from keras.layers import Input, Dense, Reshape, Flatten, Dropout, multiply, GlobalAveragePooling2D, MaxPooling2D
from keras.layers import BatchNormalization, Activation, Embedding, ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model
from keras.optimizers import Adam

import matplotlib.pyplot as plt

import numpy as np
from scipy.misc import imread
from scipy.misc import imresize
from scipy.misc import imsave
import glob
import random

from keras.preprocessing.image import ImageDataGenerator
from keras import applications
from keras.optimizers import SGD
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras.callbacks import EarlyStopping
from keras.utils import to_categorical
#from keras.applications.resnet50 import preprocess_input

from keras.applications.resnet50 import ResNet50
from keras.preprocessing import image
from keras.applications.resnet50 import preprocess_input, decode_predictions
#from keras.applications.vgg19 import preprocess_input as preprocess_vgg
from keras.applications.inception_v3 import preprocess_input as inception_preprocess

#from keras.applications.inception_resnet_v2 import preprocess_input

import numpy as np

import keras
import copy
import cv2

from keras.models import *
from keras.layers import *
from keras.optimizers import *

import numpy as np 
import os
import numpy as np
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras import backend as keras

Using TensorFlow backend.


In [2]:
path = '../data/sources/nparrs_384/'
savepath = '../data/'
images = glob.glob(path+'*.npy')
totalImages = 1800

In [3]:
def myGenerator(batch_size):
    while True:
        index_list = random.sample(range(1, totalImages), batch_size)
        alldata_x = []
        alldata_y = []
        for i in index_list:
            frame = images[i]
            frame = np.load(frame)
            tile_index = np.random.randint(0, 49)
            #print(i, tile_index, frame.shape)
            alldata_x.append(tile_index*totalImages+i)
            alldata_y.append(frame[tile_index])
        alldata_x = np.array(alldata_x)
        alldata_y = np.array(alldata_y)
        alldata_y = (alldata_y.astype(np.float32) - 127.5) / 127.5
        yield alldata_y, alldata_y
#x = myGenerator(10)
#xtrain, ytrain = next(x)
#print('xtrain shape:',xtrain.shape)
#print('ytrain shape:',ytrain.shape)

In [81]:
def myGenerator_new(batch_size):
    while True:
        index_list = random.sample(range(1, totalImages), batch_size)
        alldata_x = []
        alldata_y = []
        for i in index_list:
            frame = images[i]
            frame = np.load(frame)
            tdx = np.random.randint(0, 199)
            choices = None
            classes = {}
            if tdx < 20:
                if tdx == 0:
                    choices = [1, 20, 21]
                    classes = {1:5, 20:7, 21:8}
                elif tdx == 19:
                    choices = [18, 38, 39]
                    classes = {18:3, 38:6, 29:7}
                else:
                    choices = [tdx-1, tdx+1, tdx+19, tdx+20, tdx+21]
                    classes = {tdx-1:3, tdx+1:5, tdx+19:6, tdx+20:7, tdx+21:8}
            elif tdx > 179:
                if tdx == 199:
                    choices = [178, 179, 198]
                    classes = {178:0, 179:1, 198:3}
                elif tdx == 180:
                    choices = [160, 161, 181]
                    classes = {160:0, 161:1, 181:5}
                else:
                    choices = [tdx-21, tdx-20, tdx-19, tdx-1, tdx+1]
                    classes = {tdx-21:0, tdx-20:1, tdx-19:2, tdx-1:3, tdx+1:5}
            else:
                choices = [tdx-21, tdx-20, tdx-19, tdx-1, tdx+1, tdx+19, tdx+20, tdx+21]
                classes = {tdx-21:0, tdx-20:1, tdx-19:2, tdx-1:3, tdx+1:5, tdx+19:6, tdx+20:7, tdx+21:8}
            choice = random.choice(choices)
            cls = classes[choice]
            print(i, tdx, choice, cls, choices)
            alldata_x.append(frame[tdx])
            alldata_y.append(frame[choice])
        alldata_x = np.array(alldata_x)
        alldata_y = np.array(alldata_y)
        alldata_y = (alldata_y.astype(np.float32) - 127.5) / 127.5
        yield alldata_x, alldata_y, cls
x = myGenerator_new(2)
xtrain, ytrain, cls = next(x)
print('xtrain shape:',xtrain.shape)
print('ytrain shape:',ytrain.shape)

77 137 138 5 [116, 117, 118, 136, 138, 156, 157, 158]
952 29 50 8 [8, 9, 10, 28, 30, 48, 49, 50]
xtrain shape: (2, 192, 192, 3)
ytrain shape: (2, 192, 192, 3)


In [79]:
tdx = 1
cls = {}
cls = {tdx:2}
print(cls)

{1: 2}


In [7]:
class CGAN():
    def __init__(self):
        # Input shape
        self.img_rows = 384
        self.img_cols = 384
        self.channels = 3
        self.img_shape = (self.img_rows, self.img_cols, self.channels)
        self.optimizer = Adam(0.0002, 0.5)

        # Build and compile the discriminator
        #self.discriminator = self.build_discriminator()
        #self.discriminator.compile(loss=['mse'],
        #    optimizer=optimizer,
        #    metrics=['accuracy'])
        
        #print(self.discriminator.summary())
        
        # Build the generator
        self.generator = self.build_generator()
        
        print(self.generator.summary())
        
        noise = Input(shape=(384, 384, 3))
        img = self.generator(noise)

        # For the combined model we will only train the generator
        #self.discriminator.trainable = False

        # The discriminator takes generated image as input and determines validity
        # and the label of that image
        #valid = self.discriminator(img)
        
        # The combined model  (stacked generator and discriminator)
        # Trains generator to fool discriminator
        #self.combined = Model(noise, [img, valid])
        #self.combined.compile(loss=['mse', 'mse'],
        #    loss_weights=[0.9, 0.1],
        #    optimizer=optimizer)

    def build_generator(self):
        input_size = (384,384,3)
        inputs = Input(input_size)

        conv1_a = Conv2D(16, 3, padding = 'same', kernel_initializer = 'he_normal')(inputs)
        conv1_a = LeakyReLU(alpha=0.2)(conv1_a)
        conv1_a = BatchNormalization(momentum=0.8)(conv1_a)

        conv1_a = Conv2D(16, 3, padding = 'valid', kernel_initializer = 'he_normal')(conv1_a)
        conv1_a = LeakyReLU(alpha=0.2)(conv1_a)
        conv1_a = BatchNormalization(momentum=0.8)(conv1_a)

        pool1_a = Conv2D(16, 3, subsample=(2, 2), kernel_initializer = 'he_normal')(conv1_a)
        pool1_a = LeakyReLU(alpha=0.2)(pool1_a)
        pool1_a = BatchNormalization(momentum=0.8)(pool1_a)
        pool1_a = ZeroPadding2D(padding=(1, 1))(pool1_a)



        conv1 = Conv2D(32, 3, padding = 'same', kernel_initializer = 'he_normal')(pool1_a)
        conv1 = LeakyReLU(alpha=0.2)(conv1)
        conv1 = BatchNormalization(momentum=0.8)(conv1)

        conv1 = Conv2D(32, 3, padding = 'valid', kernel_initializer = 'he_normal')(conv1)
        conv1 = LeakyReLU(alpha=0.2)(conv1)
        conv1 = BatchNormalization(momentum=0.8)(conv1)

        pool1 = Conv2D(32, 3, subsample=(2, 2), kernel_initializer = 'he_normal')(conv1)
        pool1 = LeakyReLU(alpha=0.2)(pool1)
        pool1 = BatchNormalization(momentum=0.8)(pool1)



        conv2 = Conv2D(64, 3, padding = 'same', kernel_initializer = 'he_normal')(pool1)
        conv2 = LeakyReLU(alpha=0.2)(conv2)
        conv2 = BatchNormalization(momentum=0.8)(conv2)

        conv2 = Conv2D(64, 3, padding = 'same', kernel_initializer = 'he_normal')(conv2)
        conv2 = LeakyReLU(alpha=0.2)(conv2)
        conv2 = BatchNormalization(momentum=0.8)(conv2)

        pool2 = Conv2D(64, 3, subsample=(2, 2), kernel_initializer = 'he_normal')(conv2)
        pool2 = LeakyReLU(alpha=0.2)(pool2)
        pool2 = BatchNormalization(momentum=0.8)(pool2)



        conv2_a = Conv2D(64, 3, padding = 'same', kernel_initializer = 'he_normal')(pool2)
        conv2_a = LeakyReLU(alpha=0.2)(conv2_a)
        conv2_a = BatchNormalization(momentum=0.8)(conv2_a)

        conv2_a = Conv2D(64, 3, padding = 'same', kernel_initializer = 'he_normal')(conv2_a)
        conv2_a = LeakyReLU(alpha=0.2)(conv2_a)
        conv2_a = BatchNormalization(momentum=0.8)(conv2_a)

        pool3   = Conv2D(64, 3, subsample=(2, 2), kernel_initializer = 'he_normal')(conv2_a)
        pool3   = LeakyReLU(alpha=0.2)(pool3)
        pool3   = BatchNormalization(momentum=0.8)(pool3)



        conv3 = Conv2D(64, 3, padding = 'same', kernel_initializer = 'he_normal')(pool3)
        conv3 = LeakyReLU(alpha=0.2)(conv3)
        conv3 = BatchNormalization(momentum=0.8)(conv3)

        conv3 = Conv2D(1, 3, padding = 'valid', kernel_initializer = 'he_normal')(conv3)
        conv3 = LeakyReLU(alpha=0.2)(conv3)
        conv3 = BatchNormalization(momentum=0.8)(conv3)


        conv3_pool = Conv2D(1, 3, subsample=(2, 2), kernel_initializer = 'he_normal')(conv3)
        conv3_pool = LeakyReLU(alpha=0.2)(conv3_pool)
        conv3_pool = BatchNormalization(momentum=0.8)(conv3_pool)

        # #Decoder

        conv3_unpool = UpSampling2D(size = (2,2))(conv3_pool)
        conv3_unpool = ZeroPadding2D(padding=(1, 1))(conv3_unpool)

        conv3_unpool = Conv2D(1, 3, padding = 'same', kernel_initializer = 'he_normal')(conv3_unpool)
        conv3_unpool = LeakyReLU(alpha=0.2)(conv3_unpool)
        conv3_unpool = BatchNormalization(momentum=0.8)(conv3_unpool)


        up1    = UpSampling2D(size = (2,2))(conv3_unpool)
        #up1    = UpSampling2D(size = (2,2))(conv3)
        up1    = ZeroPadding2D(padding=(3, 3))(up1)


        conv4 = Conv2D(64, 3, padding = 'same', kernel_initializer = 'he_normal')(up1)
        conv4 = LeakyReLU(alpha=0.2)(conv4)
        conv4 = BatchNormalization(momentum=0.8)(conv4)

        merge1 = concatenate([conv4, conv2_a], axis = 3)

        conv5 = Conv2D(64, 3, padding = 'same', kernel_initializer = 'he_normal')(merge1)
        conv5 = LeakyReLU(alpha=0.2)(conv5)
        conv5 = BatchNormalization(momentum=0.8)(conv5)

        conv5 = Conv2D(64, 3, padding = 'same', kernel_initializer = 'he_normal')(conv5)
        conv5 = LeakyReLU(alpha=0.2)(conv5)
        conv5 = BatchNormalization(momentum=0.8)(conv5)


        up2    = UpSampling2D(size = (2,2))(conv5)
        up2    = ZeroPadding2D(padding=(1, 1))(up2)

        conv6 = Conv2D(64, 3, padding = 'same', kernel_initializer = 'he_normal')(up2)
        conv6 = LeakyReLU(alpha=0.2)(conv6)
        conv6 = BatchNormalization(momentum=0.8)(conv6)

        merge2 = concatenate([conv2, conv6], axis = 3)

        conv7 = Conv2D(64, 3, padding = 'same', kernel_initializer = 'he_normal')(merge2)
        conv7 = LeakyReLU(alpha=0.2)(conv7)
        conv7 = BatchNormalization(momentum=0.8)(conv7)

        conv8 = Conv2D(64, 3, padding = 'same', kernel_initializer = 'he_normal')(conv7)
        conv8 = LeakyReLU(alpha=0.2)(conv8)
        conv8 = BatchNormalization(momentum=0.8)(conv8)

        up3    = UpSampling2D(size = (2,2))(conv8)
        up3    = ZeroPadding2D(padding=(1, 1))(up3)

        merge3 = concatenate([conv1, up3], axis = 3)
        merge3 = ZeroPadding2D(padding=(1, 1))(merge3)

        conv9 = Conv2D(32, 3, padding = 'same', kernel_initializer = 'he_normal')(merge3)
        conv9 = LeakyReLU(alpha=0.2)(conv9)
        conv9 = BatchNormalization(momentum=0.8)(conv9)

        conv10 = Conv2D(32, 3, padding = 'same', kernel_initializer = 'he_normal')(conv9)
        conv10 = LeakyReLU(alpha=0.2)(conv10)
        conv10 = BatchNormalization(momentum=0.8)(conv10)


        up4    = UpSampling2D(size = (2,2))(conv10)

        conv11 = Conv2D(16, 3, padding = 'valid', kernel_initializer = 'he_normal')(up4)
        conv11 = LeakyReLU(alpha=0.2)(conv11)
        conv11 = BatchNormalization(momentum=0.8)(conv11)

        merge4 = concatenate([conv1_a, conv11], axis = 3)
        merge4 = ZeroPadding2D(padding=(1, 1))(merge4)

        conv12 = Conv2D(16, 3, padding = 'same', kernel_initializer = 'he_normal')(merge4)
        conv12 = LeakyReLU(alpha=0.2)(conv12)
        conv12 = BatchNormalization(momentum=0.8)(conv12)

        conv13 = Conv2D(16, 3, padding = 'same', kernel_initializer = 'he_normal')(conv12)
        conv13 = LeakyReLU(alpha=0.2)(conv13)
        conv13 = BatchNormalization(momentum=0.8)(conv13)


        conv14 = Conv2D(3, 1, padding = 'same', kernel_initializer = 'he_normal', activation='tanh')(conv13)
        
        model  = Model(input = inputs, output = conv14)
        return model
    
    def build_autoencoder(self):
        self.generator.compile(loss=['mse'],optimizer=self.optimizer)
    
    def train_generator_autoencoder(self, epochs, batch_size=128, sample_interval=10):
        for epoch in range(epochs):
            X_train, y_train = next(myGenerator(batch_size))
            g_loss = self.generator.train_on_batch(X_train, X_train)
            print ("Epoch ", epoch, " G loss ", g_loss)
            if epoch % sample_interval == 0:
                self.sample_images(epoch)
                self.generator.save_weights(savepath+'weights/generator_weights_'+str(epoch)+'.h5')
            
    def build_discriminator(self):
        img   = Input(shape=(384, 384, 3))
        
        model = Sequential()
        model.add(Conv2D(64, (3, 3), input_shape=(192, 192, 3)))
        model.add(LeakyReLU(alpha=0.2))
        #model.add(Dropout(0.25))
        model.add(Conv2D(64,  (3, 3),  strides=(2, 2)))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        #model.add(Dropout(0.25))
        model.add(Conv2D(64, (3, 3)))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        #model.add(Dropout(0.25))
        model.add(Conv2D(64, (6, 6),  strides=(2, 2)))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        #model.add(Dropout(0.25))
        model.add(Conv2D(64, (3, 3),  strides=(2, 2)))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Flatten())
        
        model.add(Dense(100))
        model.add(LeakyReLU(alpha=0.2))
        
        model.add(Dense(32))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dense(1))
        
        output    = model(img)
        
        model3 = Model(img, output)
        return model3

    def train(self, epochs, batch_size=128, sample_interval=50):
        random.seed(10)
        
        # Load the dataset
        for epoch in range(epochs):
            X_train, y_train = next(myGenerator(batch_size))
            
            # Adversarial ground truths
            valid = np.ones((batch_size, 1))
            fake  = np.zeros((batch_size, 1))
            
            # ---------------------
            #  Train Discriminator
            # ---------------------
            
            # Generate a half batch of new images
            gen_imgs = self.generator.predict(X_train)

            # Train the discriminator
            d_loss_real = self.discriminator.train_on_batch(X_train, valid)
            d_loss_fake = self.discriminator.train_on_batch(gen_imgs, fake)
            d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

            # ---------------------
            #  Train Generator
            # ---------------------

            # Train the generator
            g_loss = self.combined.train_on_batch(X_train, [X_train, valid])

            # Plot the progress
            print ("%d [D loss: %f, mean_acc: %.2f%% real_acc: %.2f%% fake_acc: %.2f%%] [G loss: %f, mse: %f]" % (epoch, d_loss[0], 100*d_loss[1], 100*d_loss_real[1], 100*d_loss_fake[1], g_loss[0], g_loss[1]))

            # If at save interval => save generated image samples
            if epoch % sample_interval == 0:
                self.sample_images(epoch)
                self.generator.save_weights(savepath+'weights/generator_weights_'+str(epoch)+'.h5')
                self.discriminator.save_weights(savepath+'weights/discriminator_weights_'+str(epoch)+'.h5')

    def sample_images(self, epoch):
        r, c             = 1, 10
        X_train, y_train = next(myGenerator(10))
        gen_imgs         = self.generator.predict(X_train)
        
        # Rescale images 0 - 1
        temp     = (0.5 * gen_imgs + 0.5)*255
        gen_imgs = temp.astype(int)
        
        combined = np.array([gen_imgs[0], gen_imgs[1], gen_imgs[2], gen_imgs[3], gen_imgs[4], gen_imgs[5], gen_imgs[6], gen_imgs[7], gen_imgs[8], gen_imgs[9]])
        combined = np.hstack(combined.reshape(10, 384,384, 3))
        imsave(savepath+"images/"+str(epoch)+".png", combined)
        
        combined = np.array([X_train[0], X_train[1], X_train[2], X_train[3], X_train[4], X_train[5], X_train[6], X_train[7], X_train[8], X_train[9]])
        combined = np.hstack(combined.reshape(10, 384,384, 3))
        imsave(savepath+"images/"+str(epoch)+"_real.png", combined)

In [ ]:
cgan = CGAN()
cgan.build_autoencoder()
cgan.train_generator_autoencoder(1000000, 10, 100)

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:52: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(16, 3, kernel_initializer="he_normal", strides=(2, 2))`
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:67: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, 3, kernel_initializer="he_normal", strides=(2, 2))`
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:81: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, 3, kernel_initializer="he_normal", strides=(2, 2))`
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:95: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, 3, kernel_initializer="he_normal", strides=(2, 2))`
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:110: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(1, 3, kernel_initializer="he_normal", strides=(2, 2))`
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            (None, 384, 384, 3)  0                                            
__________________________________________________________________________________________________
conv2d_57 (Conv2D)              (None, 384, 384, 16) 448         input_5[0][0]                    
__________________________________________________________________________________________________
leaky_re_lu_55 (LeakyReLU)      (None, 384, 384, 16) 0           conv2d_57[0][0]                  
__________________________________________________________________________________________________
batch_normalization_55 (BatchNo (None, 384, 384, 16) 64          leaky_re_lu_55[0][0]             
__________________________________________________________________________________________________
conv2d_58 

Epoch  0  G loss  0.92775023


/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:298: DeprecationWarning: `imsave` is deprecated!
`imsave` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imwrite`` instead.
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:302: DeprecationWarning: `imsave` is deprecated!
`imsave` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imwrite`` instead.


Epoch  1  G loss  0.762076
Epoch  2  G loss  0.6759145
Epoch  3  G loss  0.5140097
Epoch  4  G loss  0.47990704
Epoch  5  G loss  0.5287901
Epoch  6  G loss  0.34979013
Epoch  7  G loss  0.3938152
Epoch  8  G loss  0.28621292
Epoch  9  G loss  0.36760345
Epoch  10  G loss  0.20936139
Epoch  11  G loss  0.29824403
Epoch  12  G loss  0.16488108
Epoch  13  G loss  0.17095368
Epoch  14  G loss  0.23412044
Epoch  15  G loss  0.14860867
Epoch  16  G loss  0.1776352
Epoch  17  G loss  0.14779632
Epoch  18  G loss  0.21988949
Epoch  19  G loss  0.13878618
Epoch  20  G loss  0.16224259
Epoch  21  G loss  0.12609172
Epoch  22  G loss  0.16225019
Epoch  23  G loss  0.14848916
Epoch  24  G loss  0.104255214
Epoch  25  G loss  0.12189039
Epoch  26  G loss  0.13504204
Epoch  27  G loss  0.09842115
Epoch  28  G loss  0.45242062
Epoch  29  G loss  0.17928655
Epoch  30  G loss  0.14862078
Epoch  31  G loss  0.11294465
Epoch  32  G loss  0.14486924
Epoch  33  G loss  0.11574139
Epoch  34  G loss  0.1236

...

Epoch  10626  G loss  0.010154336
Epoch  10627  G loss  0.00203591
Epoch  10628  G loss  0.0034850058
Epoch  10629  G loss  0.006004475
Epoch  10630  G loss  0.0026502616
Epoch  10631  G loss  0.0045295306
Epoch  10632  G loss  0.0026663707
Epoch  10633  G loss  0.00888282
Epoch  10634  G loss  0.0033723551
Epoch  10635  G loss  0.0052604363
Epoch  10636  G loss  0.003654392
Epoch  10637  G loss  0.005906155
Epoch  10638  G loss  0.0046654195
Epoch  10639  G loss  0.0026938966
Epoch  10640  G loss  0.0017830997
Epoch  10641  G loss  0.0019220632
Epoch  10642  G loss  0.0033833683
Epoch  10643  G loss  0.0025538646
Epoch  10644  G loss  0.0038892135
Epoch  10645  G loss  0.004007647
Epoch  10646  G loss  0.0042352066
Epoch  10647  G loss  0.00873798
Epoch  10648  G loss  0.0022808074
Epoch  10649  G loss  0.0094408
Epoch  10650  G loss  0.005191922
Epoch  10651  G loss  0.0019022428
Epoch  10652  G loss  0.0033993751
Epoch  10653  G loss  0.0019514421
Epoch  10654  G loss  0.0072216587


Epoch  10866  G loss  0.006969721
Epoch  10867  G loss  0.0041248286
Epoch  10868  G loss  0.008989959
Epoch  10869  G loss  0.0064786626
Epoch  10870  G loss  0.016318526
Epoch  10871  G loss  0.0035164268
Epoch  10872  G loss  0.014355255
Epoch  10873  G loss  0.0062822388
Epoch  10874  G loss  0.0052221394
Epoch  10875  G loss  0.012296708
Epoch  10876  G loss  0.0062685
Epoch  10877  G loss  0.013604079
Epoch  10878  G loss  0.009469177
Epoch  10879  G loss  0.0060453108
Epoch  10880  G loss  0.009048878
Epoch  10881  G loss  0.0052876472
Epoch  10882  G loss  0.0078464355
Epoch  10883  G loss  0.0044980766
Epoch  10884  G loss  0.0076604136
Epoch  10885  G loss  0.020687994
Epoch  10886  G loss  0.00419006
Epoch  10887  G loss  0.011296481
Epoch  10888  G loss  0.012162839
Epoch  10889  G loss  0.008618596
Epoch  10890  G loss  0.007365655
Epoch  10891  G loss  0.003335426
Epoch  10892  G loss  0.0042586494
Epoch  10893  G loss  0.01195354
Epoch  10894  G loss  0.006214421
Epoch  

Epoch  11104  G loss  0.008238314
Epoch  11105  G loss  0.0057205213
Epoch  11106  G loss  0.006879931
Epoch  11107  G loss  0.004167433
Epoch  11108  G loss  0.0039942022
Epoch  11109  G loss  0.0033716992
Epoch  11110  G loss  0.0034975149
Epoch  11111  G loss  0.0030059943
Epoch  11112  G loss  0.003710807
Epoch  11113  G loss  0.009357728
Epoch  11114  G loss  0.009949185
Epoch  11115  G loss  0.008502095
Epoch  11116  G loss  0.006915085
Epoch  11117  G loss  0.0036805843
Epoch  11118  G loss  0.003872205
Epoch  11119  G loss  0.004279473
Epoch  11120  G loss  0.0037089833
Epoch  11121  G loss  0.0025226767
Epoch  11122  G loss  0.003967204
Epoch  11123  G loss  0.0022245285
Epoch  11124  G loss  0.008041761
Epoch  11125  G loss  0.0021533049
Epoch  11126  G loss  0.003940513
Epoch  11127  G loss  0.005360621
Epoch  11128  G loss  0.0073579587
Epoch  11129  G loss  0.0033829263
Epoch  11130  G loss  0.0031262331
Epoch  11131  G loss  0.0050771814
Epoch  11132  G loss  0.0046459986

Epoch  11342  G loss  0.003817428
Epoch  11343  G loss  0.009988372
Epoch  11344  G loss  0.0041366676
Epoch  11345  G loss  0.0085969055
Epoch  11346  G loss  0.00628189
Epoch  11347  G loss  0.003042757
Epoch  11348  G loss  0.0033908063
Epoch  11349  G loss  0.004350385
Epoch  11350  G loss  0.004562478
Epoch  11351  G loss  0.0027877397
Epoch  11352  G loss  0.0051309015
Epoch  11353  G loss  0.012434622
Epoch  11354  G loss  0.0040096887
Epoch  11355  G loss  0.009262368
Epoch  11356  G loss  0.0051914747
Epoch  11357  G loss  0.0071155243
Epoch  11358  G loss  0.0033206765
Epoch  11359  G loss  0.007696876
Epoch  11360  G loss  0.0015189833
Epoch  11361  G loss  0.0062651476
Epoch  11362  G loss  0.0028449062
Epoch  11363  G loss  0.0066366894
Epoch  11364  G loss  0.005325212
Epoch  11365  G loss  0.002150778
Epoch  11366  G loss  0.0022286503
Epoch  11367  G loss  0.0030475287
Epoch  11368  G loss  0.004097707
Epoch  11369  G loss  0.0039738854
Epoch  11370  G loss  0.006369212

Epoch  11580  G loss  0.0055142427
Epoch  11581  G loss  0.003749657
Epoch  11582  G loss  0.009227003
Epoch  11583  G loss  0.002327453
Epoch  11584  G loss  0.0020293822
Epoch  11585  G loss  0.007288938
Epoch  11586  G loss  0.003361987
Epoch  11587  G loss  0.0044267997
Epoch  11588  G loss  0.0029240258
Epoch  11589  G loss  0.0023402004
Epoch  11590  G loss  0.002486143
Epoch  11591  G loss  0.013957217
Epoch  11592  G loss  0.004754444
Epoch  11593  G loss  0.009326267
Epoch  11594  G loss  0.0051862192
Epoch  11595  G loss  0.0031981817
Epoch  11596  G loss  0.0059647136
Epoch  11597  G loss  0.006204185
Epoch  11598  G loss  0.011935217
Epoch  11599  G loss  0.0069615142
Epoch  11600  G loss  0.008343942
Epoch  11601  G loss  0.0065667867
Epoch  11602  G loss  0.0064778193
Epoch  11603  G loss  0.0021309617
Epoch  11604  G loss  0.0024112887
Epoch  11605  G loss  0.0053113555
Epoch  11606  G loss  0.0054720617
Epoch  11607  G loss  0.0033186115
Epoch  11608  G loss  0.00287433

In [ ]:
cgan.train(10000, batch_size=10, sample_interval=100)

In [ ]:
cgan2 = CGAN()
cgan2.generator.load_weights('../data/weights/generator_weights_3000.h5')

In [ ]:
import time
for i in range(100):
    x = myGenerator(1)
    xtest, ytest = next(x)
    #print('xtrain shape:',xtest.shape)
    #print('ytrain shape:',ytest.shape)
    pred = cgan.generator.predict(xtest)
    pred = pred*127.5 + 127.5
    pred = pred.astype(int)
    print(pred.dtype)
    plt.imshow(pred[0])
    plt.show()
    ytest = ytest*127.5+127.5
    ytest = ytest.astype(int)
    plt.imshow(ytest[0])
    plt.show()
    #break
    #time.sleep(1)

In [ ]:
pred = cgan.generator.predict(xtest)